<img width="400" src="https://nomeroff.net.ua/images/logo/nomeroff_net.svg" alt="Nomeroff Net. Automatic numberplate recognition system" align="left"/>

## The example demonstrates license plate number detection.

Before runing this demo, please download models from [https://nomeroff.net.ua/models/](https://nomeroff.net.ua/models/) to **./models/** directory. 

In [1]:
# Import all necessary libraries.
import os
import cv2
import numpy as np
import sys
import json
import matplotlib.image as mpimg
from matplotlib import pyplot as plt
import warnings
import os
warnings.filterwarnings('ignore')

# change this property
NOMEROFF_NET_DIR = os.path.abspath('../../../')

# specify the path to Mask_RCNN if you placed it outside Nomeroff-net project
MASK_RCNN_DIR = os.path.join(NOMEROFF_NET_DIR, 'Mask_RCNN')

MASK_RCNN_LOG_DIR = os.path.join(NOMEROFF_NET_DIR, 'logs')
MASK_RCNN_MODEL_PATH = os.path.join(NOMEROFF_NET_DIR, "models/mask_rcnn_numberplate_0700.h5")
OPTIONS_MODEL_PATH =  os.path.join(NOMEROFF_NET_DIR, "models/numberplate_options_2019_05_20.h5")

# If you use gpu version tensorflow please change model to gpu version named like *-gpu.pb
mode = "cpu"
OCR_NP_UKR_TEXT = os.path.join(NOMEROFF_NET_DIR, "models/anpr_ocr_ua_12-{}.h5".format(mode))
OCR_NP_EU_TEXT =  os.path.join(NOMEROFF_NET_DIR, "models/anpr_ocr_eu_2-{}.h5".format(mode))
OCR_NP_RU_TEXT =  os.path.join(NOMEROFF_NET_DIR, "models/anpr_ocr_ru_3-{}.h5".format(mode))
OCR_NP_KZ_TEXT =  os.path.join(NOMEROFF_NET_DIR, "models/anpr_ocr_kz_4-{}.h5".format(mode))

sys.path.append(NOMEROFF_NET_DIR)

Also you may use .pb models for options detectors or OCR detectors. 
More about conversion and preparing for inference withit https://github.com/ria-com/nomeroff-net/blob/master/examples/inference_convert.ipynb.

In [2]:
# Import license plate recognition tools.
from NomeroffNet import  filters, RectDetector, TextDetector, OptionsDetector, Detector, textPostprocessing, textPostprocessingAsync

# Initialize npdetector with default configuration file.
nnet = Detector(MASK_RCNN_DIR, MASK_RCNN_LOG_DIR)
nnet.loadModel(MASK_RCNN_MODEL_PATH)

rectDetector = RectDetector()

optionsDetector = OptionsDetector({
    "class_region": ["xx_unknown", "eu_ua_2015", "eu_ua_2004", "eu_ua_1995", "eu", "xx_transit", "ru", "kz"]
})
optionsDetector.load(OPTIONS_MODEL_PATH)

# Initialize text detector.
textDetector = TextDetector({
    "eu_ua_2004_2015": {
        "for_regions": ["eu_ua_2015", "eu_ua_2004"],
        "model_path": OCR_NP_UKR_TEXT
    },
    "eu": {
        "for_regions": ["eu", "eu_ua_1995"],
        "model_path": OCR_NP_EU_TEXT
    },
    "ru": {
        "for_regions": ["ru"],
        "model_path": OCR_NP_RU_TEXT
    },
    "kz": {
        "for_regions": ["kz"],
        "model_path": OCR_NP_KZ_TEXT
    }
})

Using TensorFlow backend.
W0522 17:58:47.766450 139918547592512 deprecation.py:237] From /usr/local/lib64/python3.6/site-packages/keras/backend/tensorflow_backend.py:4139: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0522 17:58:52.994299 139918547592512 deprecation.py:237] From /mnt/data/www/nomeroff-net/Mask_RCNN/mrcnn/model.py:341: The name tf.log is deprecated. Please use tf.math.log instead.

W0522 17:58:53.008672 139918547592512 deprecation.py:506] From /mnt/data/www/nomeroff-net/Mask_RCNN/mrcnn/model.py:423: calling crop_and_resize_v1 (from tensorflow.python.ops.image_ops_impl) with box_ind is deprecated and will be removed in a future version.
Instructions for updating:
box_ind is deprecated, use box_indices instead
W0522 17:58:53.225779 139918547592512 deprecation.py:237] From /mnt/data/www/nomeroff-net/Mask_RCNN/mrcnn/model.py:721: The name tf.sets.set_intersection is deprecated. Please use tf.sets.intersection instead.

W0522 17:58:53.2273

In [3]:
# Walking through the ./examples/images/ directory and checking each of the images for license plates.
rootDir = '/var/www/datasets/fake-region/dnr/'

max_img_w = 1600
for dirName, subdirList, fileList in os.walk(rootDir):
    for fname in fileList:
        baseName = os.path.splitext(os.path.basename(fname))[0]
        img_path = os.path.join(dirName, fname)
        print(img_path)
        img = mpimg.imread(img_path)
        #plt.axis("off")
        #plt.imshow(img)
        #plt.show()
         
        # corect size for better speed
        img_w = img.shape[1]
        img_h = img.shape[0]
        img_w_r = 1
        img_h_r = 1
        if img_w > max_img_w:
            resized_img = cv2.resize(img, (max_img_w, int(max_img_w/img_w*img_h)))
            img_w_r = img_w/max_img_w
            img_h_r = img_h/(max_img_w/img_w*img_h)
        else:
            resized_img = img
        
        try:
            NP = nnet.detect([resized_img]) 

            # Generate image mask.
            cv_img_masks = await filters.cv_img_mask_async(NP)

            # Detect points.
            arrPoints = await rectDetector.detectAsync(cv_img_masks, outboundHeightOffset=3-img_w_r, fixGeometry=True, fixRectangleAngle=10)
            #print(arrPoints)
            arrPoints[..., 1:2] = arrPoints[..., 1:2]*img_h_r
            arrPoints[..., 0:1] = arrPoints[..., 0:1]*img_w_r

            # cut zones
            zones = await rectDetector.get_cv_zonesBGR_async(img, arrPoints)
            toShowZones = await rectDetector.get_cv_zonesRGB_async(img, arrPoints)

            # find standart
            regionIds, stateIds, countLines = optionsDetector.predict(zones)
            regionNames = optionsDetector.getRegionLabels(regionIds)
            foundNumber=0
            for zone, points, regionId in zip(toShowZones, arrPoints, regionIds):
                #plt.axis("off")
                    mpimg.imsave("/var/www/datasets/fake-region/dnr/img/{}_{}.png".format(baseName, foundNumber), zone)
                    foundNumber += 1
                    #plt.imshow(zone)
                #plt.show()


            #print(regionNames)

            # find text with postprocessing by standart  
            textArr = textDetector.predict(zones, regionNames, countLines)
            textArr = await textPostprocessingAsync(textArr, regionNames)
            #print(textArr)
        except Exception as e:
            print("Error")
            print(e)

/var/www/datasets/fake-region/dnr/12010-main.jpg
/var/www/datasets/fake-region/dnr/10214-main.jpg
/var/www/datasets/fake-region/dnr/12985-main.jpg
/var/www/datasets/fake-region/dnr/13094-main.jpg
/var/www/datasets/fake-region/dnr/9780-main.jpg
/var/www/datasets/fake-region/dnr/10119-main.jpg
/var/www/datasets/fake-region/dnr/9884-main.jpg
/var/www/datasets/fake-region/dnr/9800-main.jpg
/var/www/datasets/fake-region/dnr/13325-main.jpg
/var/www/datasets/fake-region/dnr/11429-main.jpg
/var/www/datasets/fake-region/dnr/12538-main.jpg
/var/www/datasets/fake-region/dnr/12399-main.jpg
/var/www/datasets/fake-region/dnr/10172-main.jpg
/var/www/datasets/fake-region/dnr/12859-main.jpg
/var/www/datasets/fake-region/dnr/12454-main.jpg
/var/www/datasets/fake-region/dnr/9995-main.jpg
/var/www/datasets/fake-region/dnr/12122-main.jpg
/var/www/datasets/fake-region/dnr/12488-main.jpg
/var/www/datasets/fake-region/dnr/11868-main.jpg
/var/www/datasets/fake-region/dnr/12216-main.jpg
/var/www/datasets/fake-r

/var/www/datasets/fake-region/dnr/12813-main.jpg
/var/www/datasets/fake-region/dnr/12669-main.jpg
/var/www/datasets/fake-region/dnr/12149-main.jpg
/var/www/datasets/fake-region/dnr/13214-main.jpg
/var/www/datasets/fake-region/dnr/13161-main.jpg
/var/www/datasets/fake-region/dnr/10412-main.jpg
/var/www/datasets/fake-region/dnr/12788-main.jpg
/var/www/datasets/fake-region/dnr/10357-main.jpg
/var/www/datasets/fake-region/dnr/11469-main.jpg
/var/www/datasets/fake-region/dnr/11653-main.jpg
/var/www/datasets/fake-region/dnr/10311-main.jpg
/var/www/datasets/fake-region/dnr/13472-main.jpg
/var/www/datasets/fake-region/dnr/13436-main.jpg
/var/www/datasets/fake-region/dnr/12814-main.jpg
/var/www/datasets/fake-region/dnr/13167-main.jpg
/var/www/datasets/fake-region/dnr/11958-main.jpg
/var/www/datasets/fake-region/dnr/10296-main.jpg
/var/www/datasets/fake-region/dnr/10380-main.jpg
/var/www/datasets/fake-region/dnr/11444-main.jpg
/var/www/datasets/fake-region/dnr/13400-main.jpg
/var/www/datasets/fa

/var/www/datasets/fake-region/dnr/10309-main.jpg
/var/www/datasets/fake-region/dnr/12111-main.jpg
/var/www/datasets/fake-region/dnr/13254-main.jpg
/var/www/datasets/fake-region/dnr/10090-main.jpg
/var/www/datasets/fake-region/dnr/12211-main.jpg
/var/www/datasets/fake-region/dnr/12847-main.jpg
/var/www/datasets/fake-region/dnr/11342-main.jpg
/var/www/datasets/fake-region/dnr/13155-main.jpg
/var/www/datasets/fake-region/dnr/13144-main.jpg
/var/www/datasets/fake-region/dnr/13358-main.jpg
/var/www/datasets/fake-region/dnr/11819-main.jpg
/var/www/datasets/fake-region/dnr/10064-main.jpg
/var/www/datasets/fake-region/dnr/12920-main.jpg
/var/www/datasets/fake-region/dnr/13538-main.jpg
/var/www/datasets/fake-region/dnr/13139-main.jpg
/var/www/datasets/fake-region/dnr/13264-main.jpg
/var/www/datasets/fake-region/dnr/11933-main.jpg
/var/www/datasets/fake-region/dnr/10416-main.jpg
/var/www/datasets/fake-region/dnr/11769-main.jpg
/var/www/datasets/fake-region/dnr/12710-main.jpg
/var/www/datasets/fa

/var/www/datasets/fake-region/dnr/12515-main.jpg
/var/www/datasets/fake-region/dnr/9973-main.jpg
/var/www/datasets/fake-region/dnr/10245-main.jpg
/var/www/datasets/fake-region/dnr/12625-main.jpg
/var/www/datasets/fake-region/dnr/12132-main.jpg
/var/www/datasets/fake-region/dnr/10387-main.jpg
/var/www/datasets/fake-region/dnr/12962-main.jpg
/var/www/datasets/fake-region/dnr/9876-main.jpg
/var/www/datasets/fake-region/dnr/13399-main.jpg
/var/www/datasets/fake-region/dnr/12928-main.jpg
/var/www/datasets/fake-region/dnr/13035-main.jpg
/var/www/datasets/fake-region/dnr/11837-main.jpg
/var/www/datasets/fake-region/dnr/10240-main.jpg
/var/www/datasets/fake-region/dnr/12466-main.jpg
/var/www/datasets/fake-region/dnr/13220-main.jpg
/var/www/datasets/fake-region/dnr/10005-main.jpg
/var/www/datasets/fake-region/dnr/13006-main.jpg
/var/www/datasets/fake-region/dnr/11833-main.jpg
/var/www/datasets/fake-region/dnr/12667-main.jpg
/var/www/datasets/fake-region/dnr/12624-main.jpg
/var/www/datasets/fake

/var/www/datasets/fake-region/dnr/12204-main.jpg
/var/www/datasets/fake-region/dnr/11749-main.jpg
/var/www/datasets/fake-region/dnr/11489-main.jpg
/var/www/datasets/fake-region/dnr/10443-main.jpg
/var/www/datasets/fake-region/dnr/11467-main.jpg
/var/www/datasets/fake-region/dnr/11826-main.jpg
/var/www/datasets/fake-region/dnr/12775-main.jpg
/var/www/datasets/fake-region/dnr/10413-main.jpg
/var/www/datasets/fake-region/dnr/11898-main.jpg
/var/www/datasets/fake-region/dnr/12673-main.jpg
/var/www/datasets/fake-region/dnr/12809-main.jpg
/var/www/datasets/fake-region/dnr/13467-main.jpg
/var/www/datasets/fake-region/dnr/12773-main.jpg
/var/www/datasets/fake-region/dnr/10345-main.jpg
/var/www/datasets/fake-region/dnr/10262-main.jpg
/var/www/datasets/fake-region/dnr/12805-main.jpg
/var/www/datasets/fake-region/dnr/13237-main.jpg
/var/www/datasets/fake-region/dnr/13435-main.jpg
/var/www/datasets/fake-region/dnr/10359-main.jpg
/var/www/datasets/fake-region/dnr/9860-main.jpg
/var/www/datasets/fak

...

/var/www/datasets/fake-region/dnr/img/9898-main_0.png
/var/www/datasets/fake-region/dnr/img/11749-main_0.png
/var/www/datasets/fake-region/dnr/img/13178-main_0.png
/var/www/datasets/fake-region/dnr/img/13118-main_0.png
/var/www/datasets/fake-region/dnr/img/12199-main_0.png
/var/www/datasets/fake-region/dnr/img/11778-main_0.png
/var/www/datasets/fake-region/dnr/img/9826-main_0.png
/var/www/datasets/fake-region/dnr/img/10159-main_0.png
/var/www/datasets/fake-region/dnr/img/11414-main_0.png
/var/www/datasets/fake-region/dnr/img/10508-main_0.png
/var/www/datasets/fake-region/dnr/img/11870-main_0.png
/var/www/datasets/fake-region/dnr/img/13432-main_0.png
/var/www/datasets/fake-region/dnr/img/12748-main_0.png
/var/www/datasets/fake-region/dnr/img/9944-main_0.png
/var/www/datasets/fake-region/dnr/img/9771-main_0.png
/var/www/datasets/fake-region/dnr/img/12277-main_0.png
/var/www/datasets/fake-region/dnr/img/11973-main_0.png
/var/www/datasets/fake-region/dnr/img/12617-main_0.png
/var/www/datas

/var/www/datasets/fake-region/dnr/img/10332-main_0.png
/var/www/datasets/fake-region/dnr/img/9974-main_0.png
/var/www/datasets/fake-region/dnr/img/12319-main_0.png
/var/www/datasets/fake-region/dnr/img/12226-main_0.png
/var/www/datasets/fake-region/dnr/img/12020-main_0.png
/var/www/datasets/fake-region/dnr/img/11397-main_0.png
/var/www/datasets/fake-region/dnr/img/12715-main_0.png
/var/www/datasets/fake-region/dnr/img/13327-main_0.png
/var/www/datasets/fake-region/dnr/img/12385-main_0.png
/var/www/datasets/fake-region/dnr/img/11902-main_0.png
/var/www/datasets/fake-region/dnr/img/12783-main_0.png
/var/www/datasets/fake-region/dnr/img/12585-main_0.png
/var/www/datasets/fake-region/dnr/img/13400-main_0.png
/var/www/datasets/fake-region/dnr/img/12682-main_0.png
/var/www/datasets/fake-region/dnr/img/12707-main_0.png
/var/www/datasets/fake-region/dnr/img/10396-main_0.png
/var/www/datasets/fake-region/dnr/img/12022-main_1.png
/var/www/datasets/fake-region/dnr/img/11868-main_0.png
/var/www/da

/var/www/datasets/fake-region/dnr/img/10225-main_0.png
/var/www/datasets/fake-region/dnr/img/11523-main_0.png
/var/www/datasets/fake-region/dnr/img/12712-main_0.png
/var/www/datasets/fake-region/dnr/img/12190-main_0.png
/var/www/datasets/fake-region/dnr/img/10072-main_0.png
/var/www/datasets/fake-region/dnr/img/10255-main_0.png
/var/www/datasets/fake-region/dnr/img/10015-main_0.png
/var/www/datasets/fake-region/dnr/img/13465-main_0.png
/var/www/datasets/fake-region/dnr/img/10431-main_0.png
/var/www/datasets/fake-region/dnr/img/12418-main_0.png
/var/www/datasets/fake-region/dnr/img/13424-main_0.png
/var/www/datasets/fake-region/dnr/img/10498-main_0.png
/var/www/datasets/fake-region/dnr/img/11600-main_0.png
/var/www/datasets/fake-region/dnr/img/12953-main_0.png
/var/www/datasets/fake-region/dnr/img/12559-main_0.png
/var/www/datasets/fake-region/dnr/img/12805-main_0.png
/var/www/datasets/fake-region/dnr/img/12059-main_0.png
/var/www/datasets/fake-region/dnr/img/11810-main_0.png
/var/www/d

/var/www/datasets/fake-region/dnr/img/12827-main_0.png
/var/www/datasets/fake-region/dnr/img/12528-main_0.png
/var/www/datasets/fake-region/dnr/img/11799-main_0.png
/var/www/datasets/fake-region/dnr/img/11781-main_0.png
/var/www/datasets/fake-region/dnr/img/12703-main_0.png
/var/www/datasets/fake-region/dnr/img/11684-main_0.png
/var/www/datasets/fake-region/dnr/img/10451-main_0.png
/var/www/datasets/fake-region/dnr/img/9886-main_0.png
/var/www/datasets/fake-region/dnr/img/11661-main_0.png
/var/www/datasets/fake-region/dnr/img/12282-main_2.png
/var/www/datasets/fake-region/dnr/img/10509-main_0.png
/var/www/datasets/fake-region/dnr/img/12025-main_0.png
/var/www/datasets/fake-region/dnr/img/12908-main_0.png
/var/www/datasets/fake-region/dnr/img/13461-main_0.png
/var/www/datasets/fake-region/dnr/img/11670-main_0.png
/var/www/datasets/fake-region/dnr/img/10041-main_1.png
/var/www/datasets/fake-region/dnr/img/11309-main_0.png
/var/www/datasets/fake-region/dnr/img/13312-main_0.png
/var/www/da

/var/www/datasets/fake-region/dnr/img/12348-main_0.png
/var/www/datasets/fake-region/dnr/img/12489-main_0.png
/var/www/datasets/fake-region/dnr/img/12462-main_0.png
/var/www/datasets/fake-region/dnr/img/9992-main_0.png
/var/www/datasets/fake-region/dnr/img/12777-main_0.png


In [4]:
# Walking through the ./examples/images/ directory and checking each of the images for license plates.
rootDir = '/var/www/datasets/fake-region/lnr/'

max_img_w = 1600
for dirName, subdirList, fileList in os.walk(rootDir):
    for fname in fileList:
        baseName = os.path.splitext(os.path.basename(fname))[0]
        img_path = os.path.join(dirName, fname)
        print(img_path)
        img = mpimg.imread(img_path)
        #plt.axis("off")
        #plt.imshow(img)
        #plt.show()
         
        # corect size for better speed
        img_w = img.shape[1]
        img_h = img.shape[0]
        img_w_r = 1
        img_h_r = 1
        if img_w > max_img_w:
            resized_img = cv2.resize(img, (max_img_w, int(max_img_w/img_w*img_h)))
            img_w_r = img_w/max_img_w
            img_h_r = img_h/(max_img_w/img_w*img_h)
        else:
            resized_img = img
        
        try:
            NP = nnet.detect([resized_img]) 

            # Generate image mask.
            cv_img_masks = await filters.cv_img_mask_async(NP)

            # Detect points.
            arrPoints = await rectDetector.detectAsync(cv_img_masks, outboundHeightOffset=3-img_w_r, fixGeometry=True, fixRectangleAngle=10)
            #print(arrPoints)
            arrPoints[..., 1:2] = arrPoints[..., 1:2]*img_h_r
            arrPoints[..., 0:1] = arrPoints[..., 0:1]*img_w_r

            # cut zones
            zones = await rectDetector.get_cv_zonesBGR_async(img, arrPoints)
            toShowZones = await rectDetector.get_cv_zonesRGB_async(img, arrPoints)

            # find standart
            regionIds, stateIds, countLines = optionsDetector.predict(zones)
            regionNames = optionsDetector.getRegionLabels(regionIds)
            foundNumber=0
            for zone, points, regionId in zip(toShowZones, arrPoints, regionIds):
                #plt.axis("off")
                    mpimg.imsave("/var/www/datasets/fake-region/lnr/img/{}_{}.png".format(baseName, foundNumber), zone)
                    foundNumber += 1
                    #plt.imshow(zone)
                #plt.show()


            #print(regionNames)

            # find text with postprocessing by standart  
            textArr = textDetector.predict(zones, regionNames, countLines)
            textArr = await textPostprocessingAsync(textArr, regionNames)
            #print(textArr)
        except Exception as e:
            print("Error")
            print(e)

/var/www/datasets/fake-region/lnr/4504-main.jpg
/var/www/datasets/fake-region/lnr/4624-main.jpg
/var/www/datasets/fake-region/lnr/4344-main.jpg
/var/www/datasets/fake-region/lnr/7312-main.jpg
/var/www/datasets/fake-region/lnr/5249-main.jpg
/var/www/datasets/fake-region/lnr/5695-main.jpg
/var/www/datasets/fake-region/lnr/4460-main.jpg
/var/www/datasets/fake-region/lnr/7680-main.jpg
/var/www/datasets/fake-region/lnr/6130-main.jpg
/var/www/datasets/fake-region/lnr/4409-main.jpg
/var/www/datasets/fake-region/lnr/5263-main.jpg
/var/www/datasets/fake-region/lnr/4572-main.jpg
/var/www/datasets/fake-region/lnr/6268-main.jpg
/var/www/datasets/fake-region/lnr/6094-main.jpg
/var/www/datasets/fake-region/lnr/5784-main.jpg
/var/www/datasets/fake-region/lnr/6998-main.jpg
/var/www/datasets/fake-region/lnr/5215-main.jpg
/var/www/datasets/fake-region/lnr/5743-main.jpg
/var/www/datasets/fake-region/lnr/7334-main.jpg
/var/www/datasets/fake-region/lnr/7732-main.jpg
/var/www/datasets/fake-region/lnr/4780-m

/var/www/datasets/fake-region/lnr/5051-main.jpg
/var/www/datasets/fake-region/lnr/7659-main.jpg
/var/www/datasets/fake-region/lnr/6798-main.jpg
/var/www/datasets/fake-region/lnr/4179-main.jpg
/var/www/datasets/fake-region/lnr/4497-main.jpg
/var/www/datasets/fake-region/lnr/4807-main.jpg
/var/www/datasets/fake-region/lnr/4132-main.jpg
/var/www/datasets/fake-region/lnr/6516-main.jpg
/var/www/datasets/fake-region/lnr/5751-main.jpg
/var/www/datasets/fake-region/lnr/6265-main.jpg
/var/www/datasets/fake-region/lnr/6539-main.jpg
/var/www/datasets/fake-region/lnr/6213-main.jpg
/var/www/datasets/fake-region/lnr/4502-main.jpg
/var/www/datasets/fake-region/lnr/5638-main.jpg
/var/www/datasets/fake-region/lnr/5696-main.jpg
/var/www/datasets/fake-region/lnr/6423-main.jpg
/var/www/datasets/fake-region/lnr/7233-main.jpg
/var/www/datasets/fake-region/lnr/5612-main.jpg
/var/www/datasets/fake-region/lnr/6470-main.jpg
/var/www/datasets/fake-region/lnr/6663-main.jpg
/var/www/datasets/fake-region/lnr/5916-m

/var/www/datasets/fake-region/lnr/6276-main.jpg
/var/www/datasets/fake-region/lnr/4937-main.jpg
/var/www/datasets/fake-region/lnr/5590-main.jpg
/var/www/datasets/fake-region/lnr/7430-main.jpg
/var/www/datasets/fake-region/lnr/4835-main.jpg
/var/www/datasets/fake-region/lnr/4820-main.jpg
/var/www/datasets/fake-region/lnr/6655-main.jpg
/var/www/datasets/fake-region/lnr/4661-main.jpg
/var/www/datasets/fake-region/lnr/5086-main.jpg
/var/www/datasets/fake-region/lnr/5777-main.jpg
/var/www/datasets/fake-region/lnr/6376-main.jpg
/var/www/datasets/fake-region/lnr/4567-main.jpg
/var/www/datasets/fake-region/lnr/4468-main.jpg
/var/www/datasets/fake-region/lnr/6494-main.jpg
/var/www/datasets/fake-region/lnr/7341-main.jpg
/var/www/datasets/fake-region/lnr/7667-main.jpg
/var/www/datasets/fake-region/lnr/7406-main.jpg
/var/www/datasets/fake-region/lnr/5800-main.jpg
/var/www/datasets/fake-region/lnr/5427-main.jpg
/var/www/datasets/fake-region/lnr/6876-main.jpg
/var/www/datasets/fake-region/lnr/7183-m

/var/www/datasets/fake-region/lnr/7149-main.jpg
/var/www/datasets/fake-region/lnr/5324-main.jpg
/var/www/datasets/fake-region/lnr/5574-main.jpg
/var/www/datasets/fake-region/lnr/5828-main.jpg
/var/www/datasets/fake-region/lnr/5035-main.jpg
/var/www/datasets/fake-region/lnr/5346-main.jpg
/var/www/datasets/fake-region/lnr/4331-main.jpg
/var/www/datasets/fake-region/lnr/6228-main.jpg
/var/www/datasets/fake-region/lnr/5887-main.jpg
/var/www/datasets/fake-region/lnr/6510-main.jpg
/var/www/datasets/fake-region/lnr/5022-main.jpg
/var/www/datasets/fake-region/lnr/5935-main.jpg
/var/www/datasets/fake-region/lnr/6903-main.jpg
/var/www/datasets/fake-region/lnr/4276-main.jpg
/var/www/datasets/fake-region/lnr/4829-main.jpg
/var/www/datasets/fake-region/lnr/4861-main.jpg
/var/www/datasets/fake-region/lnr/6224-main.jpg
/var/www/datasets/fake-region/lnr/5395-main.jpg
/var/www/datasets/fake-region/lnr/4574-main.jpg
/var/www/datasets/fake-region/lnr/7266-main.jpg
/var/www/datasets/fake-region/lnr/7651-m

/var/www/datasets/fake-region/lnr/6580-main.jpg
/var/www/datasets/fake-region/lnr/4121-main.jpg
/var/www/datasets/fake-region/lnr/6646-main.jpg
/var/www/datasets/fake-region/lnr/5378-main.jpg
/var/www/datasets/fake-region/lnr/4327-main.jpg
/var/www/datasets/fake-region/lnr/5020-main.jpg
/var/www/datasets/fake-region/lnr/7134-main.jpg
/var/www/datasets/fake-region/lnr/5386-main.jpg
/var/www/datasets/fake-region/lnr/5433-main.jpg
/var/www/datasets/fake-region/lnr/7492-main.jpg
/var/www/datasets/fake-region/lnr/7708-main.jpg
/var/www/datasets/fake-region/lnr/7512-main.jpg
/var/www/datasets/fake-region/lnr/5040-main.jpg
/var/www/datasets/fake-region/lnr/7489-main.jpg
/var/www/datasets/fake-region/lnr/6781-main.jpg
/var/www/datasets/fake-region/lnr/5594-main.jpg
/var/www/datasets/fake-region/lnr/4932-main.jpg
/var/www/datasets/fake-region/lnr/4349-main.jpg
/var/www/datasets/fake-region/lnr/7802-main.jpg
/var/www/datasets/fake-region/lnr/6235-main.jpg
/var/www/datasets/fake-region/lnr/5126-m

...

/var/www/datasets/fake-region/lnr/img/6176-main_0.png
/var/www/datasets/fake-region/lnr/img/7298-main_0.png
/var/www/datasets/fake-region/lnr/img/7824-main_3.png
/var/www/datasets/fake-region/lnr/img/7628-main_0.png
/var/www/datasets/fake-region/lnr/img/6229-main_0.png
/var/www/datasets/fake-region/lnr/img/4623-main_0.png
/var/www/datasets/fake-region/lnr/img/7523-main_0.png
/var/www/datasets/fake-region/lnr/img/4227-main_0.png
/var/www/datasets/fake-region/lnr/img/4647-main_0.png
/var/www/datasets/fake-region/lnr/img/4259-main_0.png
/var/www/datasets/fake-region/lnr/img/7265-main_0.png
/var/www/datasets/fake-region/lnr/img/5528-main_0.png
/var/www/datasets/fake-region/lnr/img/5456-main_0.png
/var/www/datasets/fake-region/lnr/img/6204-main_0.png
/var/www/datasets/fake-region/lnr/img/6419-main_0.png
/var/www/datasets/fake-region/lnr/img/6387-main_0.png
/var/www/datasets/fake-region/lnr/img/6824-main_0.png
/var/www/datasets/fake-region/lnr/img/7319-main_0.png
/var/www/datasets/fake-regio

/var/www/datasets/fake-region/lnr/img/6007-main_0.png
/var/www/datasets/fake-region/lnr/img/7502-main_0.png
/var/www/datasets/fake-region/lnr/img/6574-main_0.png
/var/www/datasets/fake-region/lnr/img/6123-main_0.png
/var/www/datasets/fake-region/lnr/img/4649-main_0.png
/var/www/datasets/fake-region/lnr/img/5946-main_0.png
/var/www/datasets/fake-region/lnr/img/7402-main_0.png
/var/www/datasets/fake-region/lnr/img/7121-main_0.png
/var/www/datasets/fake-region/lnr/img/6423-main_0.png
/var/www/datasets/fake-region/lnr/img/5716-main_0.png
/var/www/datasets/fake-region/lnr/img/7064-main_1.png
/var/www/datasets/fake-region/lnr/img/4454-main_0.png
/var/www/datasets/fake-region/lnr/img/5069-main_0.png
/var/www/datasets/fake-region/lnr/img/4230-main_1.png
/var/www/datasets/fake-region/lnr/img/4178-main_1.png
/var/www/datasets/fake-region/lnr/img/6857-main_0.png
/var/www/datasets/fake-region/lnr/img/4181-main_0.png
/var/www/datasets/fake-region/lnr/img/7814-main_0.png
/var/www/datasets/fake-regio

/var/www/datasets/fake-region/lnr/img/6606-main_1.png
/var/www/datasets/fake-region/lnr/img/7838-main_0.png
/var/www/datasets/fake-region/lnr/img/6079-main_1.png
/var/www/datasets/fake-region/lnr/img/6834-main_0.png
/var/www/datasets/fake-region/lnr/img/6321-main_0.png
/var/www/datasets/fake-region/lnr/img/5225-main_0.png
/var/www/datasets/fake-region/lnr/img/6274-main_0.png
/var/www/datasets/fake-region/lnr/img/5151-main_0.png
/var/www/datasets/fake-region/lnr/img/7754-main_0.png
/var/www/datasets/fake-region/lnr/img/6476-main_0.png
/var/www/datasets/fake-region/lnr/img/4556-main_0.png
/var/www/datasets/fake-region/lnr/img/5406-main_0.png
/var/www/datasets/fake-region/lnr/img/7579-main_0.png
/var/www/datasets/fake-region/lnr/img/6674-main_0.png
/var/www/datasets/fake-region/lnr/img/7798-main_0.png
/var/www/datasets/fake-region/lnr/img/7671-main_1.png
/var/www/datasets/fake-region/lnr/img/4493-main_0.png
/var/www/datasets/fake-region/lnr/img/5780-main_0.png
/var/www/datasets/fake-regio

/var/www/datasets/fake-region/lnr/img/5919-main_0.png
/var/www/datasets/fake-region/lnr/img/4980-main_1.png
/var/www/datasets/fake-region/lnr/img/6227-main_0.png
/var/www/datasets/fake-region/lnr/img/7562-main_0.png
/var/www/datasets/fake-region/lnr/img/6385-main_0.png
/var/www/datasets/fake-region/lnr/img/4976-main_0.png
/var/www/datasets/fake-region/lnr/img/4912-main_0.png
/var/www/datasets/fake-region/lnr/img/4966-main_0.png
/var/www/datasets/fake-region/lnr/img/7255-main_0.png
/var/www/datasets/fake-region/lnr/img/6655-main_0.png
/var/www/datasets/fake-region/lnr/img/7594-main_0.png
/var/www/datasets/fake-region/lnr/img/5396-main_0.png
/var/www/datasets/fake-region/lnr/img/7295-main_0.png
/var/www/datasets/fake-region/lnr/img/6833-main_0.png
/var/www/datasets/fake-region/lnr/img/6235-main_0.png
/var/www/datasets/fake-region/lnr/img/4894-main_0.png
/var/www/datasets/fake-region/lnr/img/6472-main_0.png
/var/www/datasets/fake-region/lnr/img/6661-main_0.png
/var/www/datasets/fake-regio

/var/www/datasets/fake-region/lnr/img/5503-main_0.png
/var/www/datasets/fake-region/lnr/img/4371-main_1.png
/var/www/datasets/fake-region/lnr/img/5440-main_0.png
/var/www/datasets/fake-region/lnr/img/5086-main_0.png
/var/www/datasets/fake-region/lnr/img/5119-main_0.png
/var/www/datasets/fake-region/lnr/img/7335-main_0.png
/var/www/datasets/fake-region/lnr/img/5786-main_0.png
/var/www/datasets/fake-region/lnr/img/4834-main_0.png
/var/www/datasets/fake-region/lnr/img/6177-main_0.png
/var/www/datasets/fake-region/lnr/img/5605-main_0.png
/var/www/datasets/fake-region/lnr/img/6744-main_0.png
